In [1]:
import pickle
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from tqdm import tqdm

# ─── Load Data ───────────────────────────────────────────────────────────────
dataset_path_dict = {
    "embeddings": "/home/maria/Documents/HuggingMouseData/MouseViTEmbeddings/google_vit-base-patch16-224_embeddings_logits.pkl",
    "neural": "/home/maria/LuckyMouse2/pixel_transformer_neuro/data/processed/hybrid_neural_responses.npy"  # trial-wise binary spikes
}

with open(dataset_path_dict['embeddings'], "rb") as f:
    embeddings_raw = pickle.load(f)
embeddings = embeddings_raw['natural_scenes']  # shape: (118, 1000)
print("Full embedding shape:", embeddings.shape)

neural_data = np.load(dataset_path_dict["neural"])  # shape: (neurons, 5900)
print("Neural data shape:", neural_data.shape)

# ─── Construct Trial-wise Design Matrix ──────────────────────────────────────
n_images = embeddings.shape[0]        # 118
n_trials = 50
n_total = n_images * n_trials         # 5900

X_all = np.repeat(embeddings, n_trials, axis=0)  # shape: (5900, 1000)

# ─── Split Train/Test ────────────────────────────────────────────────────────
n_train_images = 100
n_test_images = 18

train_mask = np.isin(np.tile(np.arange(n_images), n_trials), np.arange(n_train_images))
test_mask = np.isin(np.tile(np.arange(n_images), n_trials), np.arange(n_train_images, n_images))

X_train = X_all[train_mask]
X_test = X_all[test_mask]
print("Train trials:", X_train.shape[0], "| Test trials:", X_test.shape[0])

# ─── Fit Model for Each Neuron ───────────────────────────────────────────────
n_neurons = neural_data.shape[0]
r2_test_scores = []

for i in tqdm(range(n_neurons), desc="Fitting neurons"):
    y_all = neural_data[i]
    y_train = y_all[train_mask]
    y_test = y_all[test_mask]

    try:
        model = LogisticRegression(solver="liblinear", max_iter=1000)
        model.fit(X_train, y_train)
        probs_pred = model.predict_proba(X_test)[:, 1]
        r2 = r2_score(y_test, probs_pred)
        r2_test_scores.append(r2)

        print(f"Neuron {i}: R² (probs) = {r2:.4f}")
    except Exception as e:
        print(f"Neuron {i}: Error - {e}")
        r2_test_scores.append(None)

# Optional: Save R² scores
#np.save("logreg_r2_trialwise_all_neurons.npy", r2_test_scores)


Full embedding shape: (118, 1000)
Neural data shape: (39209, 5900)
Train trials: 5000 | Test trials: 900


Fitting neurons:   0%|          | 1/39209 [00:02<30:09:06,  2.77s/it]

Neuron 0: R² (probs) = -0.0013


Fitting neurons:   0%|          | 2/39209 [00:04<22:16:23,  2.05s/it]

Neuron 1: R² (probs) = -0.0106


...

Neuron 292: R² (probs) = 0.1763


Fitting neurons:   1%|          | 294/39209 [14:28<33:45:36,  3.12s/it]

Neuron 293: R² (probs) = 0.0178


Fitting neurons:   1%|          | 294/39209 [14:32<32:04:56,  2.97s/it]


KeyboardInterrupt: 

In [ ]:
np.save("logreg_r2_trialwise_all_neurons.npy", r2_test_scores)